In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys_analysis.tpscompare import load_and_normalise_mephysto
from pymedphys_dicom.dicom import depth_dose, profile

In [ ]:
ROOT_DIR = Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons")

INTERNAL_DICOM_DIR = ROOT_DIR.joinpath(r"Beam Models\Photon Monte Carlo with Flattening Filter")
EXTERNAL_DICOM_DIR = next(ROOT_DIR.glob(r"Beam Models\External *\DICOM Dose Exports"))
MEASUREMENTS_DIR = ROOT_DIR.joinpath(r"Measurements\RCCC\Photons\With Flattening Filter")
RESULTS = ROOT_DIR.joinpath(r"Results\RCCC\compare")

In [ ]:
absolute_dose_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('AbsoluteDose.csv'), index_col=0)
absolute_dose = absolute_dose_table['d10 @ 90 SSD']['6 MV']
absolute_dose

In [ ]:
output_factors = pd.read_csv(MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
absolute_doses = {
    key: output_factors[key]['6 MV'] * absolute_dose
    for key in output_factors.columns
}

absolute_doses

In [ ]:
absolute_scans_per_field = load_and_normalise_mephysto(
    MEASUREMENTS_DIR, r'06MV (\d\dx\d\d) Open\.mcc', absolute_doses, 100)

getter = operator.itemgetter('displacement', 'dose')

In [ ]:
keys = absolute_scans_per_field.keys()

In [ ]:
def load_dicom_files(directory, keys):
    dicom_file_map = {
        key: directory.joinpath(f'06MV_{key}.dcm')
        for key in keys
    }
    
    dicom_dataset_map = {
        key: pydicom.read_file(str(dicom_file_map[key]), force=True)
        for key in keys
    }
    
    return dicom_dataset_map

In [ ]:
internal_dicom_dataset_map = load_dicom_files(INTERNAL_DICOM_DIR, keys)
external_dicom_dataset_map = load_dicom_files(EXTERNAL_DICOM_DIR, keys)

In [ ]:
internal_dicom_plan = pydicom.read_file(str(INTERNAL_DICOM_DIR.joinpath('06MV_plan.dcm')), force=True)
external_dicom_plan = pydicom.read_file(str(EXTERNAL_DICOM_DIR.joinpath('06MV_plan.dcm')), force=True)

In [ ]:
def plot_one_axis(ax, displacement, meas_dose, tps_dose):
    diff = tps_dose - meas_dose
    
    lines = []
    
    lines += ax.plot(displacement, meas_dose, label='Measured Dose')
    lines += ax.plot(displacement, tps_dose, label='TPS Dose')
    ax.set_ylabel('Dose (Gy / 100 MU)')
    
    x_bounds = [np.min(displacement), np.max(displacement)]
    ax.set_xlim(x_bounds)

    ax_twin = ax.twinx()

    lines += ax_twin.plot(displacement, diff, color='C3', alpha=0.5, label='Residuals [TPS - Meas]')
    ax_twin.plot(x_bounds, [0, 0], '--', color='C3', lw=0.5)
    ax_twin.set_ylabel('Dose difference [TPS - Meas] (Gy / 100 MU)')

    labels = [l.get_label() for l in lines]
    
    ax.legend(lines, labels)
    
    return ax_twin



def plot_tps_meas_diff(displacement, meas_dose, internal_tps_dose, external_tps_dose):
    fig, ax = plt.subplots(1, 2, figsize=(16,6), sharey=True)
    ax[1].yaxis.set_tick_params(which='both', labelbottom=True)

    ax_twin = list()
    
    ax_twin.append(plot_one_axis(ax[0], displacement, meas_dose, internal_tps_dose))
    ax_twin.append(plot_one_axis(ax[1], displacement, meas_dose, external_tps_dose))
    
    ax_twin[1].get_shared_y_axes().join(ax_twin[1], ax_twin[0])
    ax_twin[1].autoscale(axis='y')
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4, top=0.86)
    
    return fig, ax


def plot_pdd_diff(key):
    depth, meas_dose = getter(absolute_scans_per_field[key]['depth_dose'])
    internal_tps_dose = depth_dose(depth, internal_dicom_dataset_map[key], internal_dicom_plan) / 10
    external_tps_dose = depth_dose(depth, external_dicom_dataset_map[key], external_dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(depth, meas_dose, internal_tps_dose, external_tps_dose)
    fig.suptitle(f'Depth Dose Comparisons | {key} field', fontsize="x-large")
    ax[0].set_title("Internal Beam Model")
    ax[1].set_title("External Beam Model")
    

for key in keys:
    plot_pdd_diff(key)
    filename = RESULTS.joinpath(f'{key}_pdd.png')
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_profile_diff(key, depth, direction):
    displacement, meas_dose = getter(absolute_scans_per_field[key]['profiles'][depth][direction])
    
    internal_tps_dose = profile(displacement, depth, direction, internal_dicom_dataset_map[key], internal_dicom_plan) / 10
    external_tps_dose = profile(displacement, depth, direction, external_dicom_dataset_map[key], external_dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(displacement, meas_dose, internal_tps_dose, external_tps_dose)
    fig.suptitle(f'{direction.capitalize()} Profile Comparisons | {key} field | Depth: {depth} mm', fontsize="x-large")
    ax[0].set_title("Internal Beam Model")
    ax[1].set_title("External Beam Model")
    

for key in keys:
    depths = absolute_scans_per_field[key]['profiles'].keys()
    for depth in depths:
        for direction in ['inplane', 'crossplane']:
            plot_profile_diff(key, depth, direction)
            filename = RESULTS.joinpath(f'{key}_profile_{depth}mm_{direction}.png')
            plt.savefig(filename)
            plt.show()